In [ ]:
from dotenv import load_dotenv
import os
from elasticsearch import helpers  # For bulk Data Uploading
from elasticsearch import Elasticsearch  # Base function for interacting with Elasticsearch

load_dotenv()
client = Elasticsearch("http://localhost:9200/", api_key=os.getenv('apikey'))

#test client
print(client.info())

### Creating the dictionnaries containing all the mappings

We read the mappings list from the metadata.csv file, which is generated externally from the shared excel file, and the list is then transformed into a dictionnary of mappings with the proper formatting.

In [1]:
import os
from pprint import pprint
from read_csv import *

# grab mappings list from the csv file, which should be in the same directory as the script
DIR_NAME = os.getcwd()
FILE_NAME = "metadata.csv"
FILE_PATH = os.path.join(DIR_NAME, FILE_NAME)
data, header = read_csv(FILE_PATH)

# create the dictionnary of mappings
mappings = create_mapping(data, header)
pprint(mappings)

{'Publication/reference': {'type': 'keyword'},
 'Tfloor': {'meta': {'file': '.in'}, 'type': 'float'},
 'alpha_rad': {'meta': {'file': '.in'}, 'type': 'float'},
 'code hash': {'meta': {'file': 'header.txt'}, 'type': 'keyword'},
 'dtmax': {'meta': {'file': '.in'}, 'type': 'float'},
 'eccentricity': {'meta': {'file': '.setup'}, 'type': 'float'},
 'equation of state': {'meta': {'file': '.in'}, 'type': 'integer'},
 'excitation_HI': {'meta': {'file': '.in'}, 'type': 'integer'},
 'f_acc': {'meta': {'file': '.in'}, 'type': 'float'},
 'icompanion_star': {'meta': {'file': '.setup'}, 'type': 'integer'},
 'icool_method': {'meta': {'file': '.in'}, 'type': 'integer'},
 'icooling': {'meta': {'file': '.in'}, 'type': 'integer'},
 'idust_opacity': {'meta': {'file': '.in'}, 'type': 'integer'},
 'iget_tdust': {'meta': {'file': '.in'}, 'type': 'integer'},
 'init wind velocity': {'meta': {'file': '.in', 'units': 'km/s'},
                        'type': 'float'},
 'isink_radiation': {'meta': {'file': '.in'},

Add other settings to the index parameters

In [2]:
"""
"settings" is technically not needed if we are working on a simple local host, but can be changed to optimise search performance on a database that is hosted on a cluster and searched by multiple users.
"mappings" is required if you wish to explicitly map fields to specific values
"""

index_definition = {
    "settings": {
        "number_of_shards": 1,
    },
    "mappings": {"properties": mappings},
}

Create the index using the parameters set above.

In [3]:
""" For now we use a test index, but the name of the index should be changed later on"""

client.indices.create(index="test_index", body=index_definition)

NameError: name 'client' is not defined

# Kibana

Once the index is created in elasticsearch, in order to view it in Kibana, go to **Management** and on left menu bar, scroll down to the **Kibana** subsection, and click **Data Views**. From here, on the top right, click **Create data view** in order to integrate the new index into the Kibana interface. This will allow you to view how kibana interprets the index you have created.

### Load information from .setup and .in file




In [ ]:
from pprint import pprint
from load_ini import *

# Define directories and file names here
DIR = "/Users/camille/Documents/runs/phantom/wind"  # Careful, this is a local path - change it to your own
PREFIX = "wind"
MODELS = ["v10e00", "v20e00", "v05e00"]

In [9]:
# Add 1 model
testData = LoadSetupData(os.path.join(DIR,MODELS[0]),PREFIX,index_definition)
client.index(index='test_index', body=testData)
del testData


 ERROR: No wind.setup file found!



SystemExit: 

/Users/camille/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
# Add multiple models
base_command = {"_index": "test_index", "_op_type": "index"}
operations = []
for model in MODELS:
    modelData = LoadSetupData(os.path.join(DIR, model), PREFIX, index_definition)
    operations.append((base_command | {"_source": modelData}))
    del modelData
print(operations)
helpers.bulk(client, operations, refresh=True)
del operations


 ERROR: No wind.setup file found!



SystemExit: 

/Users/camille/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Now look into Dashboards: Analytics - Dashboards to visualise data
Documentation: https://www.elastic.co/guide/en/kibana/current/create-a-dashboard-of-panels-with-web-server-data.html

